# Decorator Pattern vs Decorator in Python
- **The Decorator Pattern**: A design pattern that allows behavior to be added to an individual object, dynamically, without affecting the behavior of other objects from the same class. [Decorator pattern from WikiPedia][1]

[1]: <https://en.wikipedia.org/wiki/Decorator_pattern> "Decorator"
- **Decorator in Python**: They are functions which modify the functionality of other functions. They help to make our code shorter and more Pythonic. 


## 1. Basic decorator

In [1]:
from functools import wraps
# Add a, b. We will add a logging functionality to it using decorator
def add(a, b):
    return a + b

# Decorator
def log(func):
    @wraps(func)
    def decorator(*arg):
        print("logging: Start add {} and {}".format(arg[0], arg[1]))
        return func(*arg)
    return decorator

In [2]:
print("Result before decorating: {}".format(add(1,2)))

Result before decorating: 3


In [3]:
@log
def add(a, b):
    return a + b

In [4]:
print("Result after decorating: {}".format(add(1,2)))

logging: Start add 1 and 2
Result after decorating: 3


### 1.1. @ is a semantic sugar for Decorator
- Decorating function by passing it as an object
        
        def func():
            # do something here
        func = decorator(func)
- Effectively equal to above
        
        @decorator
        def func():
            # do something here

### 1.2. Why use @wraps from functools
- <https://stackoverflow.com/questions/308999/what-does-functools-wraps-do>

## 2. Decorator with parameter

In [5]:
def add(a, b):
    return a + b

print("Result before decorating: {}".format(add(1,2)))

Result before decorating: 3


In [6]:
# Decorator kept logging functionality 
# plus a customized logger name parameter
def log(logger_name="default"):
    def inner(func):
        @wraps(func)
        def decorator(*arg):
            print("{} logger: Start add {} and {}".\
                  format(logger_name, arg[0], arg[1]))
            return func(*arg)
        return decorator
    return inner

# Decorating with default parameter value
@log()
def add(a, b):
    return a + b

print("Result after decorating: {}".format(add(1,2)))

default logger: Start add 1 and 2
Result after decorating: 3


In [7]:
# Decorating with customized parameter parameter "__Main__"
@log(logger_name="__Main__")
def add(a, b):
    return a + b

print("Result after decorating: {}".format(add(1,2)))

__Main__ logger: Start add 1 and 2
Result after decorating: 3


## 3. Decorating use class decorator
### 3.0. Basic example (`__call__`)

In [8]:
class decorator(object):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        print('Called {func} with args: {args}'.format(func=self.func.__name__, args=args))
        return self.func(*args)

@decorator
def func(x,y):
    return x,y

if __name__ == '__main__':
    func(1,2)

Called func with args: (1, 2)


### 3.1. Property Example (`__get__`, `__set__`)

In [9]:
class property_(object):
    def __init__(self, func):
        self.func = func
        self.name = func.__name__

    def __get__(self, instance, cls):
        print(
            'Called property from {instance} ',
            'of {klass}'.format(instance=instance, klass=cls)
        )
        return self.func(instance)

    def __set__(self, obj, value):
        print(
            'Setting up {value} '
            'for {obj}'.format(value=value, obj=obj)
        )
        [setattr(obj, k, v) for k, v in value.items()]


class Apple(object):

    @property_
    def get_color(self):
        print('Accessing get_color property')
        return 'red'

if __name__ == '__main__':
    apple = Apple()
    print(apple.get_color)
    apple.get_color = {'shape':'triangle'}
    print(apple.shape)

Called property from {instance}  of <class '__main__.Apple'>
Accessing get_color property
red
Setting up {'shape': 'triangle'} for <__main__.Apple object at 0x7ffa74b68e48>
triangle


## 4. Decorating class (decorating all functions in a class at once)

In [10]:
import time
import datetime                 

def time_this(original_function):      
    print("decorating")
    def new_function(*args, **kwargs):
        print("starting timer")
        before = datetime.datetime.now()                     
        x = original_function(*args, **kwargs)                
        after = datetime.datetime.now()                      
        print("Elapsed Time = {0}".format(after - before))
        return x                                             
    return new_function  

def time_all_class_methods(Cls):
    class NewCls(object):
        def __init__(self,*args,**kwargs):
            self.oInstance = Cls(*args,**kwargs)
        def __getattribute__(self,s):
            """
            this is called whenever any attribute of a NewCls object is accessed. This function first tries to 
            get the attribute off NewCls. If it fails then it tries to fetch the attribute from self.oInstance (an
            instance of the decorated class). If it manages to fetch the attribute from self.oInstance, and 
            the attribute is an instance method then `time_this` is applied.
            """
            try:    
                x = super(NewCls, self).__getattribute__(s)
            except AttributeError:      
                pass
            else:
                return x
            x = self.oInstance.__getattribute__(s)
            if type(x) == type(self.__init__):  # it is an instance method
                return time_this(x)  # this is equivalent of just decorating the method with time_this
            else:
                return x
    return NewCls

#now lets make a dummy class to test it out on:
@time_all_class_methods
class Foo(object):
    def a(self):
        print("entering a")
        time.sleep(3)
        print("exiting a")

oF = Foo()
oF.a()

decorating
starting timer
entering a
exiting a
Elapsed Time = 0:00:03.000675
